In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import pandas as pd
import json
import tweepy
 
import credentials

# Script para extraer Tweets en tiempo real 

En este script vamos a tener la capacidad de leer tweets en tiempo real que se esten produciendo en los temas que deseamos analizar

In [ ]:
# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        pass

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(credentials.consumer_key, credentials.consumer_secret)
        auth.set_access_token(credentials.access_token, credentials.access_token_secret)
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class StdOutListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          

    def on_error(self, status):
        print(status)

 
if __name__ == '__main__':
 
    # Authenticate using config.py and connect to Twitter Streaming API.
    hash_tag_list = ["coronavirus"]
    fetched_tweets_filename = "tweets_hash.json"

    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

# Lectura de los tweets 

In [32]:
tweets = []
for line in open('tweets_hash.json', 'r'):
    tweets.append(json.loads(line))

In [3]:
len(tweets)  # Tweets extraidos 

15030

In [ ]:
tweets[1].keys()

In [ ]:
tweets[15]['favourites_count']

In [ ]:
tweets[2601]['retweeted_status']['favorite_count']  # Likes

In [ ]:
tweets[2602]['user']['followers_count']  # retwett 

In [ ]:
tweets[2601]  # retwett 

In [ ]:
tweets[2678]

# Creación del Dataframe

In [207]:
etiquetas=[]
usuarios_men=[]
id_user=[]
user_name=[]
followers=[]
img=[]
language=[]
Text=[]
likes=[]
retweet=[]
id_tw=[]
fecha=[]

for i in range(len(tweets)):
    
    try:
        
        id_tw.append(tweets[i]['id'])
    
    except:
        
        id_tw.append('None')
    
    try:
        
        fecha.append(tweets[i]['created_at'])
    
    except:
        
        fecha.append('None')
    
    


    try:
        
        followers.append(tweets[i]['user']['followers_count'])
    
    except:
        
        followers.append('None')
    
    try:
        
        id_user.append(tweets[i]['user']['id'])
    
    except:
        
        id_user.append('Cupo Limite')
    
      
    try:
        user_name.append(tweets[i]['user']['screen_name'])
   
    except:
        
        user_name.append('Cupo Limite')
        
    try:
        
        img.append(tweets[i]['user']['profile_image_url_https'])
    
    except:
    
        img.append('Cupo Limite')
       
    
    try:
        
         language.append(tweets[i]['lang'])

    except:
        
          language.append('Cupo Limite')

    
    try:
        
        likes.append(tweets[i]['retweeted_status']['favorite_count'])
        
    except:
        
        likes.append('None')
        
        
    try:
        
        retweet.append(tweets[i]['retweeted_status']['retweet_count'] )
        
    except:
                     
        retweet.append('None')
    
    try:
                     
        Text.append(tweets[i]['text'])
    except:
                     
        Text.append('Cupo Limite')
    
    try:
                     
        etiquetas.append(tweets[i]['retweeted_status']['extended_tweet']['entities']['hashtags'])
    
    except:
      
        etiquetas.append('No registra')
        #pass
    try:
                     
        usuarios_men.append(tweets[i]['retweeted_status']['extended_tweet']['entities']['user_mentions'])
    
    except:
                     
        usuarios_men.append('No registra')
        
    
    

In [91]:
# Revisar hastag y usuarios mencionados 

In [208]:
data ={
    
    'ID_user':id_user,
    'User name':user_name,
    'ID_Tweet':id_tw,
    'Texto':Text,
    'Fecha':fecha,
    'Likes':likes,
    'Retweet':retweet,
    'Followers':followers,
    'Profile image':img,
    'Hastag':etiquetas,
    'User mention':usuarios_men,
    'Language':language
    }

In [209]:
len(data['Hastag'])

15030

In [210]:
len(data['Hastag'])

15030

In [211]:
#ids=tweets.apply(lambda x : x['id'])

In [212]:
data_final=pd.DataFrame(data)

In [213]:
#data_final.to_csv('Dataset_coronavirus.csv')

# Relacion de idiomas de tweets

In [214]:
data_final['Language'].value_counts().head()

en     9371
es     2637
fr     1185
und     472
it      276
Name: Language, dtype: int64

In [215]:
data_final[data_final['User mention']=='[ ]']

,ID_user,User name,ID_Tweet,Texto,Fecha,Likes,Retweet,Followers,Profile image,Hastag,User mention,Language


In [216]:
df_men=data_final[data_final['User mention']!='No registra']

In [217]:
len(df_men['User mention'][1])==0

True

In [218]:
df_men['longitud']=df_men['User mention'].apply(lambda x : len(x))

/home/alexbonella2806/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [219]:
df_men=df_men[df_men['longitud']!=0]

In [220]:
df_men=df_men.reset_index(inplace=False)
df_men=df_men.drop(columns='index')

In [221]:
df_men['User mention'][2]

[{'screen_name': 'RPerezGay',
  'name': 'Rafael Pérez Gay',
  'id': 338679358,
  'id_str': '338679358',
  'indices': [46, 56]},
 {'screen_name': '1luismartin',
  'name': 'luis martin quiñones',
  'id': 307709007,
  'id_str': '307709007',
  'indices': [79, 91]}]

In [222]:
Nombres=[]

for i in range(len(df_men)):
    
    #print(i)
    
    if (df_men['longitud'][i]=='1'):
        
        Nombres.append(df_men['User mention'][i]['screen_name'])
    
    else:
        
        for j in range(len(df_men['User mention'][i])):
            
            Nombres.append(df_men['User mention'][i][j]['screen_name'])
        

In [223]:
a=pd.Series(Nombres)

# Top 20 de usuarios mencionados en los tweets

In [224]:
a.value_counts().head(20)

WHO               75
DrTomFrieden      46
DrTedros          37
SaraCarterDC      27
sanidadgob        22
petrogustavo      20
CorteSupremaJ     20
HelenBranswell    19
DeZurdaTeam       19
DiazCanelB        18
agnesbuzyn        16
JoanTarda         16
StateDept         16
vivekagnihotri    16
AltNews           15
layzhang          15
AlvaradoC_MPPS    14
AmbCuiTiankai     13
AmbLiuXiaoMing    12
BloombergAsia     12
dtype: int64

In [225]:
esp=data_final[data_final['Language']=='es']
ingles=data_final[data_final['Language']=='en']

In [226]:
#ingles.sort_values(by='Retweet',ascending=False)

# Top 5 mas Retweets

In [227]:
ingles[ingles['Retweet']!='None'].sort_values(by='Retweet',ascending=False).head()

,ID_user,User name,ID_Tweet,Texto,Fecha,Likes,Retweet,Followers,Profile image,Hastag,User mention,Language
14503,1168498257837707264,LegendArmour,1226234213847879680,RT @menumpahkan: How to wear a mask\n#coronavi...,Sat Feb 08 19:59:53 +0000 2020,105659,82951,141,https://pbs.twimg.com/profile_images/116849861...,No registra,No registra,en
15028,2600569249,cuz_y_not2,1226234378336055297,RT @OlaTinee: Coronavirus: Africa are you read...,Sat Feb 08 20:00:32 +0000 2020,326645,80378,52,https://pbs.twimg.com/profile_images/122254213...,[],[],en
15011,1018718567758467072,StarbustingO,1226234373852336136,RT @OlaTinee: Coronavirus: Africa are you read...,Sat Feb 08 20:00:31 +0000 2020,326643,80377,112,https://pbs.twimg.com/profile_images/113306155...,[],[],en
15005,20571984,_stilldreaming_,1226234371683835905,RT @OlaTinee: Coronavirus: Africa are you read...,Sat Feb 08 20:00:31 +0000 2020,326641,80376,74,https://pbs.twimg.com/profile_images/104084792...,[],[],en
15003,536574384,ZoeLabarthe,1226234370542985217,RT @OlaTinee: Coronavirus: Africa are you read...,Sat Feb 08 20:00:30 +0000 2020,326640,80375,394,https://pbs.twimg.com/profile_images/121882452...,[],[],en


# Top 5 tweets con mas Likes

In [228]:
ingles[ingles['Retweet']!='None'].sort_values(by='Likes',ascending=False).head()

,ID_user,User name,ID_Tweet,Texto,Fecha,Likes,Retweet,Followers,Profile image,Hastag,User mention,Language
15028,2600569249,cuz_y_not2,1226234378336055297,RT @OlaTinee: Coronavirus: Africa are you read...,Sat Feb 08 20:00:32 +0000 2020,326645,80378,52,https://pbs.twimg.com/profile_images/122254213...,[],[],en
15011,1018718567758467072,StarbustingO,1226234373852336136,RT @OlaTinee: Coronavirus: Africa are you read...,Sat Feb 08 20:00:31 +0000 2020,326643,80377,112,https://pbs.twimg.com/profile_images/113306155...,[],[],en
15005,20571984,_stilldreaming_,1226234371683835905,RT @OlaTinee: Coronavirus: Africa are you read...,Sat Feb 08 20:00:31 +0000 2020,326641,80376,74,https://pbs.twimg.com/profile_images/104084792...,[],[],en
15003,536574384,ZoeLabarthe,1226234370542985217,RT @OlaTinee: Coronavirus: Africa are you read...,Sat Feb 08 20:00:30 +0000 2020,326640,80375,394,https://pbs.twimg.com/profile_images/121882452...,[],[],en
14984,2641704103,biigbryy,1226234362972295168,RT @OlaTinee: Coronavirus: Africa are you read...,Sat Feb 08 20:00:29 +0000 2020,326628,80374,236,https://pbs.twimg.com/profile_images/121588770...,[],[],en
